<a href="https://colab.research.google.com/github/AlimU11/ISQBD/blob/main/ISQBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python (warm-up)

## Check if the first and last number of a list is the same

In [ ]:
numbers_x = [10, 20, 30, 40, 10]
numbers_y = [75, 65, 35, 75, 30]

## Turn every item of a list into its square

In [ ]:
numbers = [1, 2, 3, 4, 5, 6, 7]

## Extend list1 by list2

In [ ]:
list1 = ["A", "B"]
list2 = ["C", "D"]

## Get Only unique items from two sets

In [ ]:
set1 = {10, 20, 30, 40, 50}
set2 = {30, 40, 50, 60, 70}

## Return a set of elements present in Set A or B, but not both

In [ ]:
set1 = {10, 20, 30, 40, 50}
set2 = {30, 40, 50, 60, 70}

## Create a Passive class without any variables and methods 

# CRISP-DM 

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np

# Data Mining 

In [ ]:
import urllib

file = urllib.request
file.urlretrieve('https://github.com/AlimU11/ISQBD/blob/main/customers_data.db?raw=true', 'customers_data.db')

## Fetch data from customer_data.db to variable df as DataFrame

# Data Understanding

## Show basic info about data

## Transform TotalCharges to numeric

In [ ]:
df.TotalCharges = 

## Check NA values

## Drop all NA values

## Create df1 from df without customerID

In [ ]:
df1 = 

## Convert Churn column to binary numeric variable

## Convert other non-numeric columns to dummies
## Assign result to variable df2

In [ ]:
df2 = 

## Get columns correlation 
## (Optional) plot correlation for Churn column

## Plot value counts of Contract as bar chart 

## Plot distribution of customers by their tenure

# Data Preparation

## Define X as all column from df2 except Churn column
## Define y as Churn column from df2

In [ ]:
X = 
y = 

## Scale X using MinMaxScaler or <a href='https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing'> any other</a> of your choice

## Split X_scaled and y to train and test

# Modeling

## Create LogisticRegression model
## use parameters solver='liblinear' and class_weight={0:1,1:2} inside constructor
## fit model using X_train, y_train

## Random Forest

# Create <a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn-ensemble-randomforestclassifier"> random forest classifier </a> model

## Use following parameters inside constructor
### n_estimators=1000
### oob_score = True
### n_jobs = -1
### random_state=50
### max_features = "auto"
### max_leaf_nodes=30
### class_weight={0:1,1:2}

# fit model with X_train and y_train

## XGBoost

## Create XGBClassifier model named xgb_cl from xgboost model
## Use parameter 
## use_label_encoder=False
## for constructor
## Fit model with X_train, y_train
## Use sample_weight=w_array inside fit function

In [ ]:
w_array = np.array([1]*y_train.shape[0], dtype=float)
w_array[y_train==1] = 0.90
w_array[y_train==0] = 0.10

# Evaluation

## use predict function for all models above
## Use recall_score from sklearn.metrics to compare y_test and prediction results

## Launch cells to show feature importance and confusion matrix for xgb classifier

In [ ]:
import xgboost as xgb

xgb_cl.get_booster().feature_names = list(X.columns.values)

fig, ax = plt.subplots(figsize=(16,8))
xgb.plot_importance(xgb_cl, ax=ax)

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix  
import seaborn as sns

plt.figure(figsize=(16,8))
sns.heatmap(confusion_matrix(y_test, xgb_cl.predict(X_test)), annot=True, fmt='5')

# Deployment

## Use pickle to dump model object
## dump scaler object, use '.sav' extension

##  Deployment (optional, requires installation in local environment)

In [ ]:
from werkzeug.wrappers import Request, Response
from werkzeug import run_simple
from flask import Flask, request, render_template


app = Flask(__name__, template_folder='')

model = pickle.load(open('model.pkl', 'rb'))

# scale values for prediction to the same distribution
scaler = pickle.load(open('scaler.sav', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def predict():
    temp = [i for i in request.form.values()]
    
    # Process values from request in order to transform to required for model shape
    row_dict = {column: np.array([], dtype=df1.dtypes.values[i]) for i, column in enumerate(df1.columns.values[:-1])}
    for i, column in enumerate(row_dict.keys()):
        row_dict[column] = np.append(row_dict[column], temp[i]).astype(df1.dtypes.values[i])

    # Create rows from unique values of initial data
    # The easiest way to produce the same values with pd.get_dummies for only one row data from request
    unique_dict = {}
    mask = df1.iloc[:, np.argwhere(((df1.dtypes != np.int64) & (df1.dtypes != np.float64)).values).reshape(-1)]
 
    for column in df1.columns.values[:-1]:
        unique_dict[column] = 0

    for column in mask.columns:
        unique_dict[column] = mask[column].unique()

    for i in range(mask.nunique().max()):
        for column in mask.columns:
            if len(unique_dict[column]) < mask.nunique().max():
                unique_dict[column] = np.append(unique_dict[column], unique_dict[column][0])

    unique_dict['SeniorCitizen'] = [0, 1, 0, 0]

    unique_df = pd.DataFrame.from_dict(unique_dict)
    unique_df1 = pd.concat([unique_df, pd.DataFrame.from_dict(row_dict)])
    
    # assert unique values match
    # theoretically it can fail in case it receives another categorical variables (from request combobox)
    assert np.all(np.hstack(np.array([unique_df[i].unique() for i in unique_df.loc[:, ~unique_df.columns.isin(['tenure', 'MonthlyCharges', 'TotalCharges'])].columns], dtype=object)) == 
                  np.hstack(np.array([unique_df1[i].unique() for i in unique_df1.loc[:, ~unique_df1.columns.isin(['tenure', 'MonthlyCharges', 'TotalCharges'])].columns], dtype=object)))
    
    df_dummies = pd.get_dummies(unique_df1)
    X = scaler.transform(df_dummies)
    return render_template('index.html', prediction_text = 'Churn detected' if model.predict(X[-1].reshape(1, -1))[0] else 'No churn expected')   

if __name__ == "__main__":
    run_simple('localhost', 9000, app)